In [4]:
import urllib.request as req
import json, ssl, urllib
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import bs4
import time, datetime
import mysql.connector
from mysql.connector import Error


    
reservoir_data={'新山水庫':[],'石門水庫':[],'翡翠水庫':[],'寶山水庫':[],'寶山第二水庫':[],'永和山水庫':[],'明德水庫':[],
                       '鯉魚潭水庫':[],'德基水庫':[],'石岡壩':[],'霧社水庫':[],'日月潭水庫':[],'湖山水庫':[],'仁義潭水庫':[],
                       '蘭潭水庫':[],'白河水庫':[],'烏山頭水庫':[],'曾文水庫':[],'南化水庫':[],'阿公店水庫':[],'牡丹水庫':[]}
reservoir_list=reservoir_data.keys()

#網站1
#開啟網站
driver=webdriver.Chrome('chromedriver.exe')
url="https://fhy.wra.gov.tw/reservoirpage_2011/statistics.aspx"
driver.get(url)

select_type = Select(driver.find_element_by_name("ctl00$cphMain$cboSearch"))
select_type.select_by_value("全部")
time.sleep(0.5)
#按下送出
driver.find_element_by_id("ctl00_cphMain_btnQuery").click()
time.sleep(5)

root = bs4.BeautifulSoup(driver.page_source, 'html.parser')

#解析資料
in_reservoir_list=False
tables=root.find_all("td")
for td in tables:
    if td.a !=None:
        if td.a.string in reservoir_list:
            temp_reservoir=td.a.string
            in_reservoir_list=True
            reservoir_data[temp_reservoir].append(td.a['href'][-5:])
            reservoir_data[temp_reservoir].append(td.a.string)
        else:
            in_reservoir_list=False
    else:
        if in_reservoir_list:
            reservoir_data[temp_reservoir].append(td.string)
driver.close()

#網站2              
url = 'https://data.wra.gov.tw/Service/OpenData.aspx?format=json&id=50C8256D-30C5-4B8D-9B84-2E14D5C6DF71'
context = ssl._create_unverified_context()
with req.urlopen(url, context=context) as jsondata:
    json_data = json.loads(jsondata.read().decode())
json_data = json_data['DailyOperationalStatisticsOfReservoirs_OPENDATA']
for d in json_data:
    if d['ReservoirName'] in reservoir_list:
        temp_reservoir=d['ReservoirName']
        reservoir_data[temp_reservoir].append(d["InflowVolume"])
        reservoir_data[temp_reservoir].append(d["Outflow"])
        reservoir_data[temp_reservoir].append(d["CatchmentAreaRainfall"])
        reservoir_data[temp_reservoir].append(d["EffectiveCapacity"])
        record_date=d["RecordTime"][0:10]
        
#若json中沒有水庫資料
for reservoir in reservoir_list:
    if len(reservoir_data[reservoir])<23:
        for i in range(4):
            reservoir_data[reservoir].append("0")
connection = mysql.connector.connect(
    host='ec2-3-92-133-135.compute-1.amazonaws.com', # 主機名稱
    database='reservoir_project', # 資料庫名稱
    user='databaseaws',        # 帳號
    password='databaseaws')  # 密碼   

#以石門水庫或翡翠水庫的日期資料作為統一的date(避免資料日期出錯無法寫入)
date=reservoir_data['石門水庫'][2][0:10]
for reservoir in reservoir_list:
    #檢查是否已存在資料,若存在就停止寫入動作
    sql = "SELECT * FROM `reservoir_water_condition` WHERE date= %s"
    new_data = (date,)
    cursor = connection.cursor()
    cursor.execute(sql, new_data)
    result=cursor.fetchall()
    if result:
        break
    #開始獲取需要的資料
    key=reservoir
    #水庫資料
    reservoir_id=reservoir_data[key][0]
    reservoir_name=reservoir_data[key][1]
    if reservoir_data[key][2][0:10]!='--' and reservoir_data[key][2][0:10]==date:
        date=reservoir_data[key][2][0:10]
    else:
        date=reservoir_data['翡翠水庫'][2][0:10]

    #有效蓄水量
    effective_water_storage=reservoir_data[key][7].replace(",","")
    if effective_water_storage=="--":
             effective_water_storage=0
    else:
        effective_water_storage=float(effective_water_storage)

    #蓄水百分比
    storage_percentage=(reservoir_data[key][8].strip(" %"))

    #有效容量
    #若網址1無資料則使用網址2的資料
    if (effective_water_storage=="--" or storage_percentage=="--" ):
        effective_capacity=reservoir_data[key][22].replace(",","")
        if effective_capacity=="" or effective_capacity==None:
            sql = "SELECT effective_capacity FROM `reservoir_water_condition` WHERE date=%s AND reservoir_id=%s"
            date = datetime.datetime.strptime(date, "%Y-%m-%d")
            new_data = ((date-datetime.timedelta(days=1)).strftime("%Y-%m-%d"),reservoir_id)
            cursor = connection.cursor()
            cursor.execute(sql, new_data)
            
            effective_capacity=cursor.fetchall()[0][0]
            if effective_capacity=="" or effective_capacity==[] or effective_capacity==None:
                effective_capacity=0
    #網址1有資料
    elif (storage_percentage=="0"):
        effective_capacity=0
    else:
        effective_capacity=(float(effective_water_storage)/float(storage_percentage))*100
    
    #入水量
    inflow=reservoir_data[key][19].replace(","," ")
    if inflow==None or inflow=="" or inflow=='--':
            inflow=0
    else:
        inflow=float(inflow)
    #出水量
    outflow=reservoir_data[key][20].replace(","," ")
    
    if outflow=="" or outflow==None or outflow=='--':
        outflow=0
    else:
        outflow=float(outflow)
    #雨量
    reservoir_rainfall=reservoir_data[key][21].replace(","," ")
    if reservoir_rainfall==None or reservoir_rainfall=="" or reservoir_rainfall=='--':
        reservoir_rainfall=0
    else:
        reservoir_rainfall=float(reservoir_rainfall)
    time.sleep(0.05)
    date = datetime.datetime.strptime(date, "%Y-%m-%d")
    try:
        if record_date==(date-datetime.timedelta(days=1)).strftime("%Y-%m-%d") or record_date==(date).strftime("%Y-%m-%d"):
            #print(reservoir_id,date,float(effective_water_storage),float(effective_capacity))
            date=str(date)[0:10]
            sql = "INSERT INTO reservoir_water_condition (reservoir_id,date,effective_water_storage,effective_capacity,inflow,outflow,reservoir_rainfall) VALUES (%s, %s, %s, %s, %s, %s, %s);"
            new_data = (reservoir_id,date,(effective_water_storage),(effective_capacity),(inflow),(outflow),(reservoir_rainfall))
            cursor = connection.cursor()
            cursor.execute(sql, new_data)
            
        else:
            sql = "INSERT INTO reservoir_water_condition (reservoir_id,date,effective_water_storage,effective_capacity) VALUES (%s, %s, %s, %s);"
            date=str(date)[0:10]
            new_data = (reservoir_id,date,(effective_water_storage),(effective_capacity))
            cursor = connection.cursor()
            cursor.execute(sql, new_data)
        # 資料存入資料庫
        connection.commit()
    except Error as e:
        time.sleep(0.1)
        print("資料庫寫入失敗：", e)
        print(new_data)
        #若寫入失敗資料全部刪除並結束迴圈
        try:
            command = "DELETE FROM reservoir_water_condition WHERE date= %s"
            cursor = connection.cursor()
            date=str(date)[0:10]
            cursor.execute(command, (date,))
            connection.commit()
            time.sleep(0.1)
        except Error as e:
            print("刪除失敗",e)
        break  
if (connection.is_connected()):
    cursor.close()
    connection.close()